# Merge Osquery Data from different systems
After running Osquery on different systems use this Jupyter Notebook to add the data into one big CSV file.

## Import needed modules

In [1]:
# Standard modules to use and manipulate dataframes
import numpy as np
import pandas as pd

## Variables and Functions

In [2]:
%cd ..
location = %pwd
#file location
csvlocation = location + "/Data/"
#TODO research method to automate / and \ for windows and linux

c:\Users\kwidamas\Desktop\Notebooks\OSQUERY\Osquery_Data_Graph


In [3]:
def read_csv_data(filename):
    df_r = pd.read_csv(csvlocation + filename)
    return(df_r)

## Read-in Osquery Data

In [4]:
#Read in data
data_orig = read_csv_data('data_total_dup_MAC.csv')
data_add = read_csv_data('data_total_dup_MAC.csv')

#NOFILTER
print(data_orig.shape)
print(data_add.shape)

(13309, 4)
(13309, 4)


## Create the `hash` based on same `anon` and filter them out
The `anon` data is used per DataFrame, if the DataFrame already has the `hash` columns skip it, otherwise the data will not be correct anymore. As you can get collisions with same `anon` but different `hash` because off the addition of other DataFrame to that data. 

In [5]:
if not 'hash' in data_orig:
    #groupby and create hash of connections
    dict_orig_hash = pd.util.hash_pandas_object(data_orig.groupby('anon')['Table.Column'].apply(', '.join), index=False).to_dict()
    #add hash to df
    data_orig['hash'] = data_orig['anon'].map(dict_orig_hash)
    print(data_orig.shape)

if not 'hash' in data_add:
    #groupby and create hash of connections
    dict_add_hash = pd.util.hash_pandas_object(data_add.groupby('anon')['Table.Column'].apply(', '.join), index=False).to_dict()
    #add hash to df
    data_add['hash'] = data_add['anon'].map(dict_add_hash)
    print(data_add.shape)

(13309, 5)
(13309, 5)


In [6]:
#find out hashes that are in `data_add` and not in `data_orig`, if you change the order you will get more looking at the other DF but that is not a correct view.
data_add_diff = data_add[~data_add.hash.isin(data_orig.hash)].sort_values(by='hash')
data_add_diff.shape

(0, 5)

In [7]:
data_add_diff.nunique()

Table           0
Table.Column    0
Column          0
anon            0
hash            0
dtype: int64

## Add the diff data to the source data
Drop `anon` column as it is not unique anymore.

In [8]:
FORMAT = ['Table','Table.Column','Column','hash'] #,'anon'
data_add_diff = data_add_diff[FORMAT]
data_orig = data_orig[FORMAT]
print(data_orig.shape)
print(data_add_diff.shape)
data_total = pd.concat([data_add_diff,data_orig])
print(data_total.shape)
data_total.to_csv(csvlocation + 'data_total_dup_MAC1.csv', index=False)

(13309, 4)
(0, 4)
(13309, 4)
